Copyright © 2020, SAS Institute Inc., Cary, NC, USA.  All Rights Reserved. 
SPDX-License-Identifier: Apache-2.0

# Configure a Private Docker Publishing Destination

In order to create an Private Docker destination, you must complete the following steps:

1. Find a valid domain with Private Docker credentials.
2. Create credentials for a specific user or group and define a credential domain.
3. Submit an API post to create a Private Docker destination. The delete destination API is also included.

## Establish a Connection and Define URLs

In [ ]:
import sys
sys.path.append('..')
import mmAuthorization

import requests
import json, os, pprint
import base64

In [ ]:
def list_destinations(destination_url, auth_token): 
    headers = {
        mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + auth_token
    }

    print("List the destinations...")
    try:
        response = requests.get(destination_url, headers=headers)
        jsondata = response.json()
        destinations = jsondata['items']
        if len(destinations) > 0:
            for destination in destinations:            
                print(destination["id"])
                print(destination["name"])
                print("===========")

    except:
        raise RuntimeError("ERROR: Could not get a destination list.")

public_ip = "PUBLIC_IP"
host_name = "HOST"
port = "PORT"

host_url="http://" + host_name + port
destination_url = host_url + "/modelPublish/destinations/"
modelrepo_url = host_url + "/modelRepository/models/"
publishmodel_url = host_url + "/modelPublish/models"
domains_url = host_url + "/credentials/domains"
print(host_url)

## Get an Authorization Token and Define Headers

In [ ]:
mm_auth = mmAuthorization.mmAuthorization("myAuth")

admin_userId = '<SAS_user_admin_ID>'
user_passwd = '<SAS_user_password>'

admin_auth_token = mm_auth.get_auth_token(host_url, admin_userId, user_passwd)

credential_admin_headers = {
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

credential_domain_headers = {
    "If-Match":"false",
    "Content-Type":"application/json",
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

credential_user_headers = {
    "If-Match":"false",
    "Content-Type":"application/json",
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

destination_harbor_headers = {
    "If-Match":"false",
    "Content-Type":"application/vnd.sas.models.publishing.destination.privatedocker+json",
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

print(admin_auth_token)

## Create a Domain Credential

In [ ]:
domain_name = "<Domain_Name>"
description = '<Domain_Description>'

my_domain_url = domains_url + "/" + domain_name
domain_attrs = {
    "id":domain_name,
    "type":"base64",
    "description": description
}

domain = requests.put(my_domain_url, 
                       data=json.dumps(domain_attrs), headers=credential_domain_headers)

print(domain)
pprint.pprint(domain.json())

## Create User Credentials

In [ ]:
user_credential_name = admin_userId
my_credential_url = my_domain_url + "/users/" + user_credential_name

userId = "REGISTRYID"

password = "REGISTRYPASS"
encoded_userId = str(base64.b64encode(userId.encode("utf-8")), "utf-8")
encoded_password = str(base64.b64encode(password.encode("utf-8")), "utf-8")

credential_attrs = {
    "domainId":domain_name,
    "identityType":"user",
    "identityId":user_credential_name,
    "domainType":"base64",
    "properties":{"dockerRegistryUserId":encoded_userId},
    "secrets":{"dockerRegistryPasswd":encoded_password}
}

credential = requests.put(my_credential_url, 
                       data=json.dumps(credential_attrs), headers=credential_user_headers)

print(credential)
pprint.pprint(credential.json())

## Create a Private Docker Destination

In [ ]:
# Creates a new Harbor destination, expecting a response code of 201.

dest_name = "DESTINATION"
domainName = "DOMAIN"
baseRepoUrl = "URL"

destination_attrs = {
    "name":dest_name,
    "destinationType":"privateDocker",
     "properties": [{"name": "kwsImageUrl",                
                     "value": "myserver.com/kws:latest"},
                    {"name": "credDomainId",
                    "value": domainName},
                    {"name": "baseRepoUrl",
                    "value": baseRepoUrl}
                   ]
}

destination = requests.post(destination_url, data=json.dumps(destination_attrs), headers=destination_harbor_headers)

print(destination)
pprint.pprint(destination.json())

## Delete a Destination

In [ ]:
deletedURL = destination_url + dest_name

destination = requests.delete(deletedURL, headers=credential_admin_headers)

print(deletedURL)
print(destination)
pprint.pprint(destination.json())